We have student data with marks and attendance. The goal is to calculate total marks, average percentage, topper, and class performance.

Setup PySpark

In [ ]:
import findspark
findspark.init()


In [ ]:
import findspark
findspark.init()

import os
print(os.environ.get("JAVA_HOME"))


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,when, round
from functools import reduce
import operator


spark = SparkSession.builder \
    .appName("Student Performance Project PySpark") \
    .getOrCreate()


Load Data from CSV file

In [ ]:
print("All Students : ")
df = spark.read.csv("students.csv", header=True, inferSchema=True)
df.toPandas()


Check Columns name

In [ ]:
print(df.columns)

Absent Student : 

In [ ]:
print("Absent Student in one or more Subject : ")
ab = df.filter(
    (col("EM-II") == "AB") |
    (col("PPS") == "AB") |
    (col("CHEM/PHY") == "AB") |
    (col("EG/MECH") == "AB") |
    (col("BXE/BEE") == "AB")
)
print("Count of Student :",ab.count())
ab.show(df.count(), truncate=False)

In [ ]:
print("Absent Student in all subjects : ")
ab_all = df.filter(
    (col("EM-II") == "AB") &
    (col("PPS") == "AB") &
    (col("CHEM/PHY") == "AB") &
    (col("EG/MECH") == "AB") &
    (col("BXE/BEE") == "AB")
)
print("Count of Student :",ab_all.count())
ab_all.show(df.count(), truncate=False)

Fail Student : 

In [ ]:
print("Fail Student in one or more subject : ")
fail = df.filter(
    (col("EM-II") < 28 ) |
    (col("PPS") < 28 ) |
    (col("CHEM/PHY") < 28 ) |
    (col("EG/MECH") < 28 ) |
    (col("BXE/BEE") < 28 )
)
print("Count of Student :",fail.count())
fail.show(df.count(), truncate=False)

In [ ]:
print("Fail Student In All Subject : ")
fail = df.filter(
    (col("EM-II") < 28 ) &
    (col("PPS") < 28 ) &
    (col("CHEM/PHY") < 28 ) &
    (col("EG/MECH") < 28 ) &
    (col("BXE/BEE") < 28 )
)
print("Count of Students :",fail.count())
fail.show(df.count(), truncate=False)

Replace AB student to 0 mark 

In [ ]:

subjects = ["EM-II", "PPS", "CHEM/PHY", "EG/MECH", "BXE/BEE"]

for sub in subjects:
    df = df.withColumn(sub, when(col(sub) == "AB", 0).otherwise(col(sub).cast("int")))


Total marks & Percentage of each student : 

In [ ]:
print("Total Marks & Percentage :")
df = df.withColumn("Total", reduce(operator.add, [col(sub) for sub in subjects]))

max_marks_per_subject = 70
total_max = max_marks_per_subject * len(subjects)

df = df.withColumn("Percentage", round((col("Total") / total_max) * 100, 2)) 

df.select("Roll No", "Candidate Name", "Total", "Percentage").show(df.count(), truncate=False)

In [ ]:
df.show(df.count(), truncate=False)

In [ ]:
topper = df.orderBy(col("Percentage").desc()).limit(1)

print("Topper :")
topper.show()
